<a href="https://colab.research.google.com/github/Habiba-Arafa/Machine-learning-Models-From-Scratch/blob/main/Neural_Network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Normalize X_train & X_test
X_train = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [ ]:
# Initialize weights and biases
def initialise(input_neuron, hidden_neuron, output_neuron):
    w1 = np.random.randn(hidden_neuron, input_neuron) * np.sqrt(2.0 / input_neuron)
    w2 = np.random.randn(output_neuron, hidden_neuron) * np.sqrt(2.0 / hidden_neuron)
    b1 = np.zeros((hidden_neuron, 1))
    b2 = np.zeros((output_neuron, 1))
    return w1, b1, w2, b2

In [ ]:
# Activation functions and their derivatives
def ReLU(z):
    return np.maximum(0, z)

def linear(z):
    return z

def linearPrime(z):
    return 1

def ReLUPrime(z):
    return (z > 0).astype(float)

In [ ]:
# Forward propagation
def forwardProp(a0, w1, b1, w2, b2):
    z1 = np.dot(w1, a0) + b1
    a1 = ReLU(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = linear(z2)
    return z1, a1, z2, a2

In [ ]:
# Compute gradients
def computeGradient(a2, y, z2, a1, w2, z1, a0):
    m = a2.shape[1]
    de2 = (a2 - y) * linearPrime(z2)
    db2 = de2
    dw2 = np.dot(de2, a1.T)
    da1 = np.dot(w2.T, de2)
    de1 = da1 * ReLUPrime(z1)
    db1 = de1
    dw1 = np.dot(de1, a0.T)
    return dw1, db1, dw2, db2

# Update weights and biases
def updateWeightsAndBiases(w1, b1, w2, b2, dw1, db1, dw2, db2, eta):
    w1 = w1 - eta * dw1
    b1 = b1 - eta * db1.sum(axis=1, keepdims=True)
    w2 = w2 - eta * dw2
    b2 = b2 - eta * db2.sum(axis=1, keepdims=True)
    return w1, b1, w2, b2

In [ ]:
# Training function
def train(X_train, y_train, w1, b1, w2, b2, eta, epochs, batch_size):
    n_samples = X_train.shape[0]
    for epoch in range(epochs):
        cost = 0
        for i in range(0, n_samples, batch_size):
            batch_start = i
            batch_end = min(n_samples, i + batch_size)
            X_batch = X_train[batch_start:batch_end].T
            y_batch = y_train[batch_start:batch_end].T

            z1, a1, z2, a2 = forwardProp(X_batch, w1, b1, w2, b2)
            dw1, db1, dw2, db2 = computeGradient(a2, y_batch, z2, a1, w2, z1, X_batch)
            w1, b1, w2, b2 = updateWeightsAndBiases(w1, b1, w2, b2, dw1, db1, dw2, db2, eta)

            current_cost = np.sum((a2 - y_batch) ** 2) * 0.5
            cost += current_cost

        print(f"Epoch {epoch + 1}/{epochs}, Cost: {cost}")

    return w1, b1, w2, b2

In [ ]:
# Testing function
def test(X_test, w1, b1, w2, b2):
    z1, a1, z2, a2 = forwardProp(X_test.T, w1, b1, w2, b2)
    return a2


In [ ]:
input_neuron = X_train.shape[1]
hidden_neuron = 128
output_neuron = 10

# Initialize weights and biases
w1, b1, w2, b2 = initialise(input_neuron, hidden_neuron, output_neuron)

# Set hyperparameters
eta = 0.01
epochs = 20
batch_size = 64

# Training loop
w1, b1, w2, b2 = train(X_train, y_train, w1, b1, w2, b2, eta, epochs, batch_size)

# Evaluate on test data
y_pred = test(X_test, w1, b1, w2, b2)

Epoch 1/20, Cost: 30338.306574854156
Epoch 2/20, Cost: 6902.191966040078
Epoch 3/20, Cost: 6399.976825574024
Epoch 4/20, Cost: 6134.580531096247
Epoch 5/20, Cost: 5956.378845301379
Epoch 6/20, Cost: 5779.602189366295
Epoch 7/20, Cost: 5617.844774457905
Epoch 8/20, Cost: 5489.189933102029
Epoch 9/20, Cost: 5082.153608186728
Epoch 10/20, Cost: 4687.040080387593
Epoch 11/20, Cost: 4600.6658404487725
Epoch 12/20, Cost: 4544.311694213162
Epoch 13/20, Cost: 4499.763835636283
Epoch 14/20, Cost: 4461.830936145185
Epoch 15/20, Cost: 4423.362876671967
Epoch 16/20, Cost: 4362.693004428666
Epoch 17/20, Cost: 4285.150050068615
Epoch 18/20, Cost: 4220.3257097336555
Epoch 19/20, Cost: 4167.172269587236
Epoch 20/20, Cost: 4130.0553436768605


In [ ]:
# Convert one-hot encoded y_test back to categorical labels
true_labels = np.argmax(y_test, axis=1)
# Convert predictions to categorical labels
predicted_labels = np.argmax(y_pred, axis=0)



# Compute accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels) * 100
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 93.91%
